In [ ]:
(()=>{
    
    class Broker{
        constructor(){
            this.servers={}
        }
        _add =(key, server) =>{
            this.servers[key] = server
        }
        _passTo = (s,key,prop, value) =>{
            console.log(s, key, prop, value)
            this.servers[s][key][prop] = JSON.parse(JSON.stringify(value))
        }
        _get = (s,key, prop ) =>{
            return JSON.parse(JSON.stringify(this.servers[s][key][prop]))
        }
    }
    class Realm{
        constructor(server, broker){
            this.server=server
            this.broker = broker
            this.x=function(key){
                return {
                    get: (target, prop) =>{
                        if(prop.substr(0,3)===this.server){
                            return target[prop]
                        }else{
                            let remoteServer = prop.substr(0,3)
                            return this.broker._get(remoteServer, key, prop)
                        }
                    },
                    set:(target, prop,value) =>{
                        if(prop.substr(0,3)===this.server){
                            return target[prop] = value
                        }else{
                            let remoteServer = prop.substr(0,3)
                            this.broker._passTo(remoteServer, key,prop, value)
                        }
                    }
                }
            }
            this.users= new Proxy({}, {...this.x('users')})
            this.pools= new Proxy({}, {...this.x('pools')})

        }
        
        AddUser = (name)=>{
            let userId = this.server+Math.random()
            this.users[userId] = {name: name}
            return userId
        }
        CreatePool = (masterUserId ) =>{
            let master = this.users[masterUserId]
            let poolId = this.server+Math.random()
            this.pools[poolId] = {
                users: [masterUserId],
                master: masterUserId
            }
            this.users[masterUserId].pool = poolId
            return poolId
        }
    }

    let broker = new Broker() 
    let r2 = new Realm('ah2', broker)
    let r1 = new Realm('ah1', broker)
    broker._add('ah1', r1)
    broker._add('ah2', r2)
    r1.users.ah2hassan='asdf'
//     let u1 = r1.AddUser('hassan')
//     let u2 = r2.AddUser('ansari')
    
//     let p1 = r1.CreatePool(u2)
    console.log(r1, r2)
})()

In [377]:
            (()=>{
    class Realm{
        constructor(server){
            this.server = server;
            this.newUUID = () => this.server+Math.random()
            this.keyToPropMap = new Map()
            this.map = new Map()
            this.subMap = new Map()
        }
        define = (name, schema) =>{
            this.keyToPropMap.set(name,[])
            let keyToPropMap = this.keyToPropMap.get(name)
            Object.keys(schema).forEach(key=>{
                keyToPropMap.push(key)
                this.map.set(`${name}To${key}`,new Map())
            /*
                switch(typeof(schema[key])){
                    case 'string':
                        this.map.set(`${name}To${key}`,{})
                        break;
                    case 'object':
                        let subKey = this.newUUID()
                        this.map.set(`${name}To${key}`, subKey)
                        this.subMap.set(subKey, new Map())
                        break;
                }
            */
            })
        }
        Key = (uniqueIdentifier) =>{
            this.uniqueIdentifier = uniqueIdentifier
//             this.select = 
        }
        select = (key) =>{
            if(this.key)
                key = this.key+'To'+key
            
            this.key = key
            console.log(key)
            if(this.map.has(key)){
                this.get = (key) =>{
                    return this.map.get(key)
                }
                this.set = (key, value) =>{
                    this.map.set(key, value)
                }
            }else{
                this.select = this.select
                this.create = (data) =>{
                    //some validating logic here
                    let newUUID = this.newUUID()     
                    let map, newSubUUID, subMap
                    Object.keys(data).forEach(dataKey=>{
                        switch(typeof(data[dataKey])){
                            case 'string':
                                map = this.map.get(`${key}To${dataKey}`)
                                map.set(newUUID, data[dataKey])
                                break;
                            case 'object':        
                                map = this.map.get(`${key}To${dataKey}`)
                                newSubUUID = this.newUUID()
                                map.set(newUUID, newSubUUID)
                                this.subMap.set(newSubUUID, new Map())
                                subMap = this.subMap.get(newSubUUID)
                                Object.keys(data[dataKey]).forEach(_=>{
                                    subMap.set(_, data[dataKey][_])
                                })
                                break;
                                
                        }
                    })
                    return newUUID
                }
            }
            
            return this
        }        
    }
    let db1 = new Realm('ah1')
    db1.define('user', {
        name: 'string',
        device: {
            name: 'string',
            model: 'string'
        }
    })
    let u1 = db1.select('user').create({
        name: 'hassan',
        device: {
            name: 'android',
            model: 'm10'
        }
    })
    db1.select('user')
    db1.select('user')
    db1.select('user')
                
    console.log(db1)
})()

user
userTouser
userTouserTouser
userTouserTouserTouser
Realm {
  define: [Function: define],
  Key: [Function: Key],
  select: [Function: select],
  server: 'ah1',
  newUUID: [Function (anonymous)],
  keyToPropMap: Map(1) { 'user' => [ 'name', 'device' ] },
  map: Map(2) {
    'userToname' => Map(1) { 'ah10.9984807169990018' => 'hassan' },
    'userTodevice' => Map(1) { 'ah10.9984807169990018' => 'ah10.5488716417367452' }
  },
  subMap: Map(1) {
    'ah10.5488716417367452' => Map(2) { 'name' => 'android', 'model' => 'm10' }
  },
  key: 'userTouserTouserTouser',
  create: [Function (anonymous)]
}


In [429]:
(()=>{
    class Realm{
        constructor(server){
            this.server = server;
            this.newUUID = () => this.server+Math.random()
            this.keyToPropMap = new Map()
            this.map = new Map()
            this.subMap = new Map()
            this.schema = new Map()
        }
        define = (name, schema) =>{
            this.keyToPropMap.set(name,[])
            let keyToPropMap = this.keyToPropMap.get(name)
            Object.keys(schema).forEach(key=>{
                keyToPropMap.push(key)
                this.map.set(`${name}To${key}`,new Map())
                this.schema.set(`${name}To${key}`, typeof(schema[key]))
                if(typeof(schema[key])==='object'){
                    this.keyToPropMap.set(`${name}To${key}`,[])
                    let keyToPropMap = this.keyToPropMap.get(`${name}To${key}`)                    
                    Object.keys(schema[key]).forEach(subkey=>{
                        keyToPropMap.push(subkey)
                        this.schema.set(`${name}To${key}To${subkey}`, typeof(schema[key][subkey]))
                        
                    })
                }
            })
        }
        collection = (collection) =>{
            let self ={}
            self.collection = collection
            
            self.select = (key)=>{
                if(self.key)
                    self.key = self.key+'To'+key
                else
                    self.key = self.collection+'To'+key
                if(this.map.has(self.key)){
//                     if(this.schema[self.key]==='string')// may be will use !=='object' in future
                    if(!this.keyToPropMap.has(self.key)) //that means key is an object
                        return this.map.get(self.key)
                    else
                        return this.map.get(self.key)
                        
                }
                return self
            }
            self.create = (payload) =>{
                let newUUID = this.newUUID()
                let map, newSubUUID, subMap
                Object.keys(payload).forEach(payloadkey=>{
                    switch(typeof(payload[payloadkey])){
                        case 'string':
                            map = this.map.get(`${self.collection}To${payloadkey}`)
                            map.set(newUUID, payload[payloadkey])
                            break;
                        case 'object':        
                            map = this.map.get(`${self.collection}To${payloadkey}`)
                            newSubUUID = this.newUUID()
                            map.set(newUUID, newSubUUID)
                            this.subMap.set(newSubUUID, new Map())
                            subMap = this.subMap.get(newSubUUID)
                            Object.keys(payload[payloadkey]).forEach(_=>{
                                subMap.set(_, payload[payloadkey][_])
                            })
                            break;

                    }
                })
                return newUUID
            }
            self.GetKeys = () =>{
                let prop = this.keyToPropMap.get(self.collection)[0]//choose any prop
                return [...this.map.get(`${self.collection}To${prop}`).keys()]
            }
            return self            
        }
                
    }
    let db1 = new Realm('ah1')
    db1.define('user', {
        name: 'string',
        device: {
            name: 'string',
            model: 'string'
        }
    })

    let u1 = db1.collection('user').create({
        name: 'hassan',
        device: {
            name: 'android',
            model: 'm10'
        }
    })
    let u2 = db1.collection('user').create({
        name: 'hassan',
        device: {
            name: 'android',
            model: 'm10'
        }
    })
    console.log(db1.collection('user').select('device'))
    console.log(db1)

})()

Map(2) {
  'ah10.09606318633432465' => 'ah10.4051550684423013',
  'ah10.5414992522858644' => 'ah10.09407329056346225'
}
Realm {
  define: [Function: define],
  collection: [Function: collection],
  server: 'ah1',
  newUUID: [Function (anonymous)],
  keyToPropMap: Map(2) {
    'user' => [ 'name', 'device' ],
    'userTodevice' => [ 'name', 'model' ]
  },
  map: Map(2) {
    'userToname' => Map(2) {
      'ah10.09606318633432465' => 'hassan',
      'ah10.5414992522858644' => 'hassan'
    },
    'userTodevice' => Map(2) {
      'ah10.09606318633432465' => 'ah10.4051550684423013',
      'ah10.5414992522858644' => 'ah10.09407329056346225'
    }
  },
  subMap: Map(2) {
    'ah10.4051550684423013' => Map(2) { 'name' => 'android', 'model' => 'm10' },
    'ah10.09407329056346225' => Map(2) { 'name' => 'android', 'model' => 'm10' }
  },
  schema: Map(4) {
    'userToname' => 'string',
    'userTodevice' => 'object',
    'userTodeviceToname' => 'string',
    'userTodeviceTomodel' => 'string'
  }


In [49]:
(()=>{
    class A{
        constuctor(){
            
        }
        collection = (collection) =>{
            let self ={}
            self.collection = collection
            self.id = (id)=>{
                let self2 = {}
                self2.id = id
                self2.select = (key, self3) =>{
                    if(!self3)
                        self3 = {}
                    if(!self3.id)
                        self3.id = self2.id
                    if(!self3.collection)
                        self3.collection = self.collection
                    self3.select = (key)=>{
                        return self2.select(key,self3)
                    }
                    return self3
                    
               }
               return self2
            }
//             self.select = (key)=>{ 
//                 let self2 = {}                
//                 if(self2.key)
//                     self2.key = self2.key+'To'+key
//                 else
//                     self2.key = self.collection+'To'+key
//                 self2.select = self.select
//                 return self2
//             }
            return self
            
        }
    }
    a = new A()
    let u1 = a.collection('user').id('1001')
    console.log(u1.select('name').select('hassan'))
    
        
})()

{ id: '1001', collection: 'user', select: [Function (anonymous)] }
